## Aula 5 - Processamento  de Imagens

### Classificadores 

In [1]:
import cv2
print(cv2.__version__)

import numpy as np

from matplotlib import pyplot as plt
import matplotlib.image as mpimg

%matplotlib inline

plt.style.use('seaborn')

3.4.1


In [2]:
# Carregando o Classificador
face_classifier = cv2.CascadeClassifier('classificadores/haarcascade_frontalface_default.xml')

image = cv2.imread('imagens/silvio.jpg')
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

faces = face_classifier.detectMultiScale(gray, 1.3, 5)

# When no faces detected, face_classifier returns and empty tuple
if faces is ():
     cv2.putText(image, "Rosto ausente", (50,50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 6)
    

# Desenhando retângulos nos rostos identificados
for (x,y,w,h) in faces:
    cv2.rectangle(image, (x,y), (x+w,y+h), (127,0,255), 2)
    cv2.imshow('Face Detection', image)
    
cv2.waitKey(0)    
cv2.destroyAllWindows()

In [3]:
# Utilizando capturas de video
def identificar_rosto(imagem):
    face_classifier = cv2.CascadeClassifier('classificadores/haarcascade_frontalface_default.xml')

    gray = cv2.cvtColor(imagem, cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray, 1.3, 5)

    if faces is ():
         cv2.putText(imagem, "Sorriso ausente", (50,50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 6)

    for (x,y,w,h) in faces:
        cv2.rectangle(imagem, (x,y), (x+w,y+h), (127,0,255), 2)

    return imagem

In [4]:
# Junto com o classificador de rosto, localiza o sorriso
def identificar_sorriso(imagem):
    face_classifier = cv2.CascadeClassifier('classificadores/haarcascade_frontalface_default.xml')
    smile_classifier = cv2.CascadeClassifier('classificadores/haarcascade_smile.xml')

    gray = cv2.cvtColor(imagem, cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray, 1.3, 5)

    if faces is ():
         cv2.putText(imagem, "ROSTO AUSENTE", (50,50), cv2.FONT_HERSHEY_SIMPLEX, 2, (0,0,255), 2)

    for (x,y,w,h) in faces:
        
        cv2.rectangle(imagem, (x, y), (x+w, y+h), (0, 0, 255), 2)
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = imagem[y:y+h, x:x+w]
        
        smile = smile_classifier.detectMultiScale(
            roi_gray,
            scaleFactor= 1.7,
            minNeighbors=22,
            minSize=(25, 25),
            flags=cv2.CASCADE_SCALE_IMAGE
            )

        # Set region of interest for smiles
        for (x, y, w, h) in smile:
            cv2.rectangle(roi_color, (x, y), (x+w, y+h), (255, 0, 0), 1)
            tamanho = int(w) * int(h)
            cv2.putText(imagem, "SORRISO [%s]"%(tamanho), (50,80), cv2.FONT_HERSHEY_SIMPLEX, 2, (255,0,0), 2)
            break

    return imagem

In [24]:
# Classificador gerado através do Cascade Trainger GUI, para identificar uma bola nas imagens
def identificar_bola(imagem):
    ball_classifier = cv2.CascadeClassifier('ball/classifier/cascade.xml')
  
    gray = cv2.cvtColor(imagem, cv2.COLOR_BGR2GRAY)
    balls = ball_classifier.detectMultiScale(gray, 1.3, 3)

    for (x,y,w,h) in balls:
        roi = imagem[y:y+h, x:x+w]
        area = int(w) * int(h)
        if area > 1500:
            cv2.rectangle(imagem, (x,y), (x+w,y+h), (127,0,255), 2)
            cv2.putText(imagem, "BOLA", (x,y-20), cv2.FONT_HERSHEY_PLAIN, 2, (0,0,255), 1)
        
    return imagem

In [25]:
# Utilizando o classificador para identificar o objeto em video
# Utilizar sleep para limitar os frames gerados durante o video
# time.sleep(<segundos>)import time

import time

cam_capture = cv2.VideoCapture("videos/soccer.avi")

while True:
    ret, image_frame = cam_capture.read()
    
    if ret:
        image_frame = identificar_bola(image_frame)
        cv2.imshow("Identificar bolas", image_frame)

        if cv2.waitKey(1) == 13:
            break
    else:
        break
        
cam_capture.release()
cv2.destroyAllWindows()

In [8]:
# Classificador de Pessoas
def identificar_bodies(imagem):
    body_classifier = cv2.CascadeClassifier('classificadores/haarcascade_fullbody.xml')
  
    gray = cv2.cvtColor(imagem, cv2.COLOR_BGR2GRAY)
    bodies = body_classifier.detectMultiScale(gray, 1.2, 3)

    for (x,y,w,h) in bodies:
        roi = imagem[y:y+h, x:x+w]
        area = int(w) * int(h)
        cv2.rectangle(imagem, (x,y), (x+w,y+h), (127,0,255), 2)
        cv2.putText(imagem, "PEDESTRE", (x,y-20), cv2.FONT_HERSHEY_PLAIN, 2, (0,0,255), 1)
        
    return imagem

In [9]:
# Identificador de Carros
def identificar_carros(imagem):
    cars_classifier = cv2.CascadeClassifier('classificadores/haarcascade_car.xml')
  
    gray = cv2.cvtColor(imagem, cv2.COLOR_BGR2GRAY)
    cars = cars_classifier.detectMultiScale(gray, 1.2, 3)

    for (x,y,w,h) in cars:
        roi = imagem[y:y+h, x:x+w]
        area = int(w) * int(h)
        cv2.rectangle(imagem, (x,y), (x+w,y+h), (127,0,255), 2)
        cv2.putText(imagem, "CARRO", (x,y-20), cv2.FONT_HERSHEY_PLAIN, 2, (0,0,255), 1)
        
    return imagem

In [10]:
cam_capture = cv2.VideoCapture("videos/walking.avi")

# Analisando pessoas no video
while True:
    ret, image_frame = cam_capture.read()
    
    if ret:
        image_frame = identificar_bodies(image_frame)
        cv2.imshow("Identificar bolas", image_frame)

        if cv2.waitKey(1) == 13:
            break
    else:
        break
        
cam_capture.release()
cv2.destroyAllWindows()

In [13]:
cam_capture = cv2.VideoCapture("videos/cars.avi")

# Identificando os carros no video
while True:
    ret, image_frame = cam_capture.read()
    
    if ret:
        image_frame = identificar_carros(image_frame)
        cv2.imshow("Identificar bolas", image_frame)

        if cv2.waitKey(1) == 13:
            break
    else:
        break
        
cam_capture.release()
cv2.destroyAllWindows()

## Desafio 1
Pesquise no repositório de classificadores pré-treinados de Haar, um classificador de sorriso e exiba em tela a área referente ao retângulo do sorriso detectado.  
Por se tratar de um componente de face, avalie combinar mais que um classificador.

In [14]:
# Identificador de sorrisos
def identificar_sorriso(imagem):
    face_classifier = cv2.CascadeClassifier('classificadores/haarcascade_frontalface_default.xml')
    smile_classifier = cv2.CascadeClassifier('classificadores/haarcascade_smile.xml')

    gray = cv2.cvtColor(imagem, cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray, 1.3, 5)

    if faces is ():
         cv2.putText(imagem, "ROSTO AUSENTE", (50,50), cv2.FONT_HERSHEY_SIMPLEX, 2, (0,0,255), 2)

    for (x,y,w,h) in faces:
        
        cv2.rectangle(imagem, (x, y), (x+w, y+h), (0, 0, 255), 2)
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = imagem[y:y+h, x:x+w]
        
        smile = smile_classifier.detectMultiScale(
            roi_gray,
            scaleFactor= 1.7,
            minNeighbors=22,
            minSize=(25, 25),
            flags=cv2.CASCADE_SCALE_IMAGE
            )

        # Set region of interest for smiles
        for (x, y, w, h) in smile:
            cv2.rectangle(roi_color, (x, y), (x+w, y+h), (255, 0, 0), 1)
            tamanho = int(w) * int(h)
            cv2.putText(imagem, "SORRISO [%s]"%(tamanho), (50,80), cv2.FONT_HERSHEY_SIMPLEX, 2, (255,0,0), 2)
            break

    return imagem

In [28]:
# Subindo imagem
image = cv2.imread('imagens/wolv.jpg')

new_image = identificar_sorriso(image)
cv2.imshow('Detector de Sorrisos', new_image)
    
cv2.waitKey(0)    
cv2.destroyAllWindows()

## Marcos Faciais


In [30]:
import dlib
import numpy 

predictor_68_path = "modelos/shape_predictor_68_face_landmarks.dat"
predictor_5_path = "modelos/shape_predictor_5_face_landmarks.dat"

predictor = dlib.shape_predictor(predictor_68_path)
detector = dlib.get_frontal_face_detector()


def get_landmarks(im):
    rects = detector(im, 1)

    if len(rects) > 1:
        raise Exception("Encontrado mais de uma face")
    if len(rects) == 0:
        raise Exception("Sem face")

    return numpy.matrix([[p.x, p.y] for p in predictor(im, rects[0]).parts()])

def annotate_landmarks(im, landmarks):
    im = im.copy()
    for idx, point in enumerate(landmarks):
        pos = (point[0, 0], point[0, 1])
        cv2.putText(im, str(idx), pos,
                    fontFace=cv2.FONT_HERSHEY_SCRIPT_SIMPLEX,
                    fontScale=0.4,
                    
                    color=(0, 0, 255))
        cv2.circle(im, pos, 3, color=(0, 255, 255))
    return im

def get_landmarks_multiple(im):
    rects = detector(im, 1)

    if len(rects) == 0:
        raise Exception("Sem face")
        
    landmarks = []
    for rect in rects:
        landmarks.append(numpy.matrix([[p.x, p.y] for p in predictor(im, rect).parts()]))

    return landmarks

def annotate_landmarks_multiple(im, landmarks):
    im = im.copy()
    
    for landmark in landmarks:
        for idx, point in enumerate(landmark):
            pos = (point[0, 0], point[0, 1])
            cv2.putText(im, str(idx), pos,
                        fontFace=cv2.FONT_HERSHEY_SCRIPT_SIMPLEX,
                        fontScale=0.4,

                        color=(0, 0, 255))
            cv2.circle(im, pos, 3, color=(0, 255, 255))
    return im

image = cv2.imread('imagens/silvio.jpg')
landmarks = get_landmarks_multiple(image)
image_with_landmarks = annotate_landmarks_multiple(image, landmarks)

cv2.imshow('Result', image_with_landmarks)
cv2.imwrite('imagens/silvio_landmarks.jpg',image_with_landmarks)
cv2.waitKey(0)
cv2.destroyAllWindows()

ModuleNotFoundError: No module named 'dlib'

## Desafio 2  
Construa um detector de bocejos que por meio da identificação de regiões da face calcule a área da boca e indique quando há ou não um bocejo.  
Crie um contador de bocejos e que as informações sejam exibidas em tempo real utilizando uma webcam.

In [32]:
import cv2
from matplotlib import pyplot as plt
import numpy as np

In [35]:
# Pegar o resultado no Git depois, no Lab estou sem a dlib para fazer o exercicio.